**Step 1:** Mount your Google Drive by clicking on "Mount Drive" in the Files section (panel to the left of this text.)

**Step 2:** Go to Runtime -> Change runtime type and select TPU.

**Step 3:** Create a folder in your Google Drive, and rename it to "vMalConv"

**Step 4:** Download the pre-processed training and test datasets.

In [ ]:
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_test.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_train.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/y_train.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/y_test.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/metadata.csv

--2021-10-13 20:19:25--  https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_test.dat
Resolving dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)... 52.219.106.194
Connecting to dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)|52.219.106.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1904800000 (1.8G) [application/x-www-form-urlencoded]
Saving to: ‘X_test.dat’

X_test.dat          100%[===================>]   1.77G  95.4MB/s    in 19s     

2021-10-13 20:19:45 (95.5 MB/s) - ‘X_test.dat’ saved [1904800000/1904800000]

--2021-10-13 20:19:45--  https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_train.dat
Resolving dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)... 52.219.99.74
Connecting to dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)|52.219.99.74|:443... connected.
HTTP request sent, awa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step 5:** Copy the downloaded files to vMalConv

In [ ]:
!cp /content/X_train.dat /content/drive/MyDrive/vMalConv/X_train.dat
!cp /content/X_test.dat /content/drive/MyDrive/vMalConv/X_test.dat
!cp /content/y_train.dat /content/drive/MyDrive/vMalConv/y_train.dat
!cp /content/y_test.dat /content/drive/MyDrive/vMalConv/y_test.dat
!cp /content/metadata.csv /content/drive/MyDrive/vMalConv/metadata.csv

**Step 6:** Download and install Ember:

In [ ]:
!wget https://github.com/endgameinc/ember/archive/master.zip
!unzip master.zip
!rm master.zip
!cp -r ember-master/* .
!rm -r ember-master
!pip install -r requirements.txt
!python setup.py install

--2021-10-15 00:05:30--  https://github.com/endgameinc/ember/archive/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/elastic/ember/archive/master.zip [following]
--2021-10-15 00:05:31--  https://github.com/elastic/ember/archive/master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/elastic/ember/zip/master [following]
--2021-10-15 00:05:31--  https://codeload.github.com/elastic/ember/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [   <=>              ]  11.22M  11.9MB/s    i

**Step 7:** Read vectorized features from the data files.

In [ ]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("drive/MyDrive/vMalConv/")
metadata_dataframe = ember.read_metadata("drive/MyDrive/vMalConv/")

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


**Step 7:** Get rid of rows with no labels.

In [ ]:
labelrows = (y_train != -1)
X_train = X_train[labelrows]
y_train = y_train[labelrows]

In [ ]:
import h5py
h5f = h5py.File('X_train.h5', 'w')
h5f.create_dataset('X_train', data=X_train)
h5f.close()
h5f = h5py.File('y_train.h5', 'w')
h5f.create_dataset('y_train', data=y_train)
h5f.close()

In [ ]:
!cp /content/X_train.h5 /content/drive/MyDrive/vMalConv/X_train.h5
!cp /content/y_train.h5 /content/drive/MyDrive/vMalConv/y_train.h5

> **Exercise 1:** Complete the following code to create the architecture of MalConv in Keras:

In [ ]:
def make_model():
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  feature_size=2381
  tf.compat.v1.disable_eager_execution()

  keras.backend.clear_session()
  
  # Model architecture
  from tensorflow.keras import layers
  
 ### Your code -- Define the layers of MalConv ###
  model = tf.keras.Sequential()
  model.add(layers.InputLayer(input_shape=(1,feature_size)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1500, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
  print(model.summary())
  return model

In [ ]:
model = make_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1, 2381)           0         
_________________________________________________________________
dense (Dense)                (None, 1, 1500)           3573000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1500)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              1501      
Total params: 3,574,501
Trainable params: 3,574,501
Non-trainable params: 0
_________________________________________________________________
None


**Step 8:** Partial fit the standardScaler to avoid overloading the memory:

In [ ]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
for x in range(0,600000,100000):
  mms.partial_fit(X_train[x:x+100000])

In [ ]:
X_train = mms.transform(X_train)

In [ ]:
## Reshape to create 3 channels ##
import numpy as np
X_train = np.reshape(X_train,(-1,1,2381))
y_train = np.reshape(y_train,(-1,1,1))

> **Exercise 2:** Complete the following code to train the model for 30 epochs, with a batch size of 128, and 20% validation split. 

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

save_dir = "drive/MyDrive/vMalConv/"

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

### Your code ###
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train,
                batch_size=128,
                epochs=30,
                  validation_split=.2,
                  callbacks=[callback]
                  )
# Save the weights #
model.save_weights (save_dir+'weights.h5')

# Save the model architecture #
model_json = model.to_json()
with open(save_dir+"model.json", "w") as json_file:
    json_file.write(model_json)

print("model saved.")

Train on 480000 samples, validate on 120000 samples
Epoch 1/30
480000/480000 [==============================] - ETA: 0s - loss: 7.0859 - accuracy: 0.9411 - auc_1: 0.9472 - precision_1: 0.9400

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


480000/480000 [==============================] - 47s 97us/sample - loss: 7.0859 - accuracy: 0.9411 - auc_1: 0.9472 - precision_1: 0.9400 - val_loss: 3.5622 - val_accuracy: 0.9727 - val_auc_1: 0.9778 - val_precision_1: 0.9861
Epoch 2/30
480000/480000 [==============================] - 44s 92us/sample - loss: 8.9602 - accuracy: 0.9581 - auc_1: 0.9615 - precision_1: 0.9571 - val_loss: 3.9701 - val_accuracy: 0.9820 - val_auc_1: 0.9843 - val_precision_1: 0.9895
Epoch 3/30
480000/480000 [==============================] - 44s 92us/sample - loss: 10.5864 - accuracy: 0.9635 - auc_1: 0.9670 - precision_1: 0.9628 - val_loss: 4.7558 - val_accuracy: 0.9849 - val_auc_1: 0.9873 - val_precision_1: 0.9910
Epoch 4/30
480000/480000 [==============================] - 45s 94us/sample - loss: 8.9435 - accuracy: 0.9663 - auc_1: 0.9705 - precision_1: 0.9646 - val_loss: 7.1687 - val_accuracy: 0.9814 - val_auc_1: 0.9871 - val_precision_1: 0.9869
Epoch 5/30
480000/480000 [==============================] - 45s 93

**Final Steps:** Download this Jupyter Notebook, and the saved model files.